# WISE2MBH Tutorial

In [1]:
import wise2mbh as wm
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import random
no_data = 9876543

In [2]:
allwise = Table.read('../samples/AllWISE_sample.fits')
allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
WISEAJ112428.14-363548.5,171.1172539,-36.5968225,0.2,9876543.0,Unknown,J112428.14-363548.5,171.1172539,-36.5968225,0.0427,0.0415,130.3,13.449,12.486,9.394,6.977,15.185,14.474,14.293,0.025,0.024,0.033,0.084,0.056,0.061,0.075,1716036401351015875,0000,0,1122,AAAA,160,54,42,55,2.073,1e-06
WISEAJ144229.68-435343.2,220.6236974,-43.8953408,0.3,9876543.0,Unknown,J144229.68-435343.2,220.6236974,-43.8953408,0.1285,0.1177,11.9,15.975,14.663,10.911,7.801,--,--,--,0.055,0.06,0.118,0.195,--,--,--,2205044001351034712,0000,0,011n,AABB,412,466,-34,505,--,1e-06
WISEAJ163728.75+290335.9,249.3698107,29.0599733,0.2,9876543.0,Unknown,J163728.75+290335.9,249.3698107,29.0599733,0.0692,0.0643,4.9,15.637,13.775,9.703,7.347,--,--,--,0.04,0.034,0.038,0.124,--,--,--,2485128701351042160,0000,0,0011,AAAB,-116,295,-250,278,--,2e-06
WISEAJ184321.55-621724.8,280.8398056,-62.2902233,0.2,9876543.0,Unknown,J184321.55-621724.8,280.8398056,-62.2902233,0.0722,0.0689,38.8,14.624,13.762,11.04,8.519,--,--,--,0.036,0.035,0.106,--,--,--,--,2810062101351018362,0000,0,nnnn,AAAU,--,--,--,--,--,5e-06
WISEAJ183203.33+633041.5,278.0139066,63.5115391,0.3,9876543.0,Unknown,J183203.33+633041.5,278.0139066,63.5115391,0.0662,0.0627,129.3,15.811,14.794,11.508,8.357,--,--,--,0.032,0.035,0.086,0.118,--,--,--,2774163601351028837,0000,0,11nn,AAAB,155,233,-229,248,--,7e-06
WISEAJ000517.26-341452.8,1.321926,-34.2480098,0.3,9876543.0,Unknown,J000517.26-341452.8,1.321926,-34.2480098,0.0752,0.0713,141.6,15.1,14.143,10.839,7.887,--,--,--,0.035,0.04,0.114,0.189,--,--,--,18034901351061482,0000,0,110n,AABB,289,256,-282,261,--,7e-06
WISEAJ041514.13-810745.9,63.8088863,-81.1294387,0.2,9876543.0,Unknown,J041514.13-810745.9,63.8088863,-81.1294387,0.0457,0.0449,162.3,14.499,13.543,10.429,7.558,--,--,--,0.025,0.026,0.048,0.099,--,--,--,681081801351062155,0000,0,110n,AAAA,260,66,92,69,--,8e-06
WISEAJ150206.92+781622.6,225.5288639,78.2729584,0.3,9876543.0,Unknown,J150206.92+781622.6,225.5288639,78.2729584,0.0886,0.0831,95.1,15.959,15.085,12.15,9.497,--,--,--,0.037,0.05,0.202,0.512,--,--,--,2232178701351000385,0000,0,11nn,AABC,-761,305,100,285,--,9e-06
WISEAJ050357.41-282633.0,75.989223,-28.4425115,0.2,9876543.0,Unknown,J050357.41-282633.0,75.989223,-28.4425115,0.1182,0.0847,153.4,15.102,14.292,11.284,8.529,--,--,--,0.045,0.045,0.125,0.332,--,--,--,754028801351050034,0000,0,nnnn,AABB,--,--,--,--,--,1.1e-05


Clening process of the sample from AllWISE

In [3]:
size_before_depure = len(allwise)
print(f'Size before depure: {size_before_depure}')

allowed_z = allwise[allwise['Z']!=0]                                                            #Masking of null redshifts
qph_list = allwise['qph'].tolist()                                                              #Extract quality flags
null_phot = ['X', 'Z']                                                                          #Null and uplim values in W bands
not_null_quality = [not any(letter in qph[:-1] for letter in null_phot) for qph in qph_list]    #Test quality flags

allwise = allowed_z[not_null_quality].copy()                                                    #Keep non-null values in quality

size_after_depure = len(allwise)
print(f'Size after depure: {size_after_depure}')

Size before depure: 77712
Size after depure: 77687


Detections with 'U' flag in qph do not contain errors on the WISE bands, so better to set them to a single value, in this case we'll set them to 0

In [4]:
allwise['e_W1mag'] = allwise['e_W1mag'].filled(0)                 #Replace null errors in WISE bands
allwise['e_W2mag'] = allwise['e_W2mag'].filled(0)
allwise['e_W3mag'] = allwise['e_W3mag'].filled(0)

Since this algorithm have a Monte Carlo implementation for error propagation, we recommend to take slices for big sample (rows>200), so for this tutorial, we'll use a random slice from the AllWISE sample.

In [5]:
random_indices = random.sample(range(len(allwise)), 1000)
new_allwise = allwise[random_indices]
new_allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
SDSS2406118265353955328,218.70867,26.71809,0.1611791,9876543.0,Galaxy,J143450.08+264305.1,218.7086755,26.7180885,0.0524,0.0504,150.4,13.984,13.733,11.698,8.914,15.74,15.123,14.323,0.025,0.032,0.165,0.332,0.067,0.086,0.073,2180127201351000033,0000,0,00nn,AABB,102,65,43,68,0.218,0.018495
SDSS2794583969264330752,162.59104,19.72218,0.16946661,9876543.0,Galaxy,J105021.85+194319.8,162.5910418,19.7221844,0.0616,0.0601,167.4,14.164,13.819,11.623,8.308,16.023,15.46,14.919,0.028,0.039,0.0,--,0.086,0.124,0.104,1632119601351033611,0000,0,11nn,AAUU,-169,120,-97,130,0.149,0.017044
SDSSJ093445.99+255954.3,143.6916555,25.9984307,0.263425,9876543.0,Unknown,J093445.99+255954.3,143.6916466,25.9984244,0.0674,0.0651,6.2,14.467,14.204,12.639,8.63,16.291,15.632,14.835,0.03,0.043,0.0,--,0.124,0.157,0.122,1433125701351048121,0000,0,00nn,AAUU,-111,153,398,168,0.374,0.036694
SDSSJ004436.20+212750.8,11.150844,21.4641137,0.38501,9876543.0,Unknown,J004436.20+212750.8,11.1508567,21.4641143,0.0815,0.0777,131.3,14.988,14.737,12.138,8.929,--,--,--,0.033,0.057,0.0,--,--,--,--,113121201351044540,0000,0,01nn,AAUU,173,139,-249,144,--,0.042717
SDSS1318545935088773120,243.62323,41.58074,0.13750392,-5.0,Galaxy,J161429.57+413450.6,243.623237,41.5807426,0.0459,0.0441,130.2,13.522,13.363,12.439,9.768,15.523,15.04,14.465,0.024,0.027,0.217,--,0.079,0.126,0.101,2432140801351055455,0000,5,00nn,AABU,-58,57,-58,61,0.197,0.021076
WISEAJ235351.53+002222.8,358.4647251721,0.373002826,0.14100733,9876543.0,Unknown,J235351.53+002222.8,358.4647385,0.3730054,0.0868,0.0819,136.2,14.839,14.759,12.268,8.544,16.454,15.618,15.437,0.034,0.073,0.0,--,0.117,0.14,0.168,3584100001351042759,0000,0,00nn,AAUU,457,241,-186,249,0.197,0.048818
SDSSJ111441.74+213219.3,168.673935,21.5387098,0.47053,9876543.0,Unknown,J111441.74+213219.3,168.6739367,21.5387033,0.1183,0.109,14.6,15.486,15.707,12.656,9.089,--,--,--,0.042,0.132,0.0,--,--,--,--,1678121201351050347,0000,0,0nnn,ABUU,235,340,-80,376,--,0.024024
SDSS2198976307767830528,151.22329,31.35643,0.2289271,9876543.0,Galaxy,J100453.58+312123.1,151.2232893,31.3564355,0.0594,0.0564,2.3,14.03,13.73,12.376,9.059,16.074,15.202,14.687,0.028,0.035,0.0,--,0.102,0.107,0.096,1517131801351015801,0000,0,00nn,AAUU,15,114,137,125,0.204,0.019854
PGC2578794,193.668579,58.4453641,0.14934534,-5.0,Unknown,J125440.45+582643.3,193.6685766,58.4453738,0.0456,0.044,142.6,13.569,13.389,12.433,9.53,15.381,14.889,14.13,0.024,0.026,0.0,--,0.071,0.114,0.071,1929159001351002899,0000,1,11nn,AAUU,-72,56,-117,57,0.05,0.035118


First, store the observed colors and then define the Monte Carlo arrays for W1, W2 and W3, with the same colors but now for Monte Carlo arrays

In [6]:
new_allwise['W1-W2_obs'] = new_allwise['W1mag'] - new_allwise['W2mag']
new_allwise['W2-W3_obs'] = new_allwise['W2mag'] - new_allwise['W3mag']

In [7]:
w1 = wm.array_montecarlo(new_allwise['W1mag'],new_allwise['e_W1mag'])
w2 = wm.array_montecarlo(new_allwise['W2mag'],new_allwise['e_W2mag'])
w3 = wm.array_montecarlo(new_allwise['W3mag'],new_allwise['e_W3mag'])

w1w2_obs = w1-w2
w2w3_obs = w2-w3

# K-corrections for the defined sample

Now that all initial colors are created, one can start by the first masking of the reject zone by deleting all sources with a W2-W3>4.4

In [8]:
first_reject_zone = new_allwise['W2-W3_obs']<=4.4

In [9]:
w1 = w1[first_reject_zone]
w2 = w2[first_reject_zone]
w3 = w3[first_reject_zone]

w1w2_obs = w1w2_obs[first_reject_zone]
w2w3_obs = w2w3_obs[first_reject_zone]

new_allwise = new_allwise[first_reject_zone]

It is necessary to create new arrays to store K-corrected W1abs and the K-correction factor of W1-W2 and W2-W3

In [11]:
w1w2_kcors = np.zeros(np.shape(w1w2_obs)[0])                                                
w2w3_kcors = np.zeros(np.shape(w2w3_obs)[0])
w1abs = np.zeros(np.shape(w1))

In [33]:
object_condition = (new_allwise['NED_TYPE']=='RadioS') | (new_allwise['NED_TYPE']=='QSO')
color_condition_1 = (new_allwise['W1-W2_obs']>0.8) & (new_allwise['W2-W3_obs']<2.2)
color_condition_2 = (new_allwise['W1-W2_obs']>wm.w1w2_treshold_qso(new_allwise['W2-W3_obs'])) & (new_allwise['W2-W3_obs']>=2.2) & (new_allwise['W2-W3_obs']<=4.4)

suboptimal_cond = (new_allwise['Z']>=0.5) & (new_allwise['Z']<=3) & ~object_condition & ~(color_condition_1 | color_condition_2)
nok_cond = (new_allwise['Z']>3) | object_condition | color_condition_1 | color_condition_2

optimal_sample = new_allwise[(new_allwise['Z']<0.5) & ~object_condition & ~(color_condition_1 | color_condition_2)]
suboptimal_sample = new_allwise[suboptimal_cond]                                                                            #Samples for k-correcion                
nok_sample = new_allwise[nok_cond]

To create the final quality tag, here we store the K-correction tag

In [34]:
new_allwise['K_QUALITY'] = int(0)

new_allwise['K_QUALITY'] = np.where(suboptimal_cond, int(1), new_allwise['K_QUALITY'])
new_allwise['K_QUALITY'] = np.where(nok_cond, int(2), new_allwise['K_QUALITY'])

Now we import the look-up tables from Jarrett+2023 to use

In [37]:
e_table = wm.kcorr_table('E')
l_table = wm.kcorr_table('L')
s_table = wm.kcorr_table('S')

In [40]:
identifier = np.random.choice(np.arange(10), size=3, replace=False)
for idn, sample in zip(identifier,[optimal_sample, suboptimal_sample, nok_sample]):
    if len(sample)!=0:
        if idn!=identifier[2]:                                                                  #Color k-correction for sample with z<0.5
            object_condition = (sample['NED_TYPE']=='RadioS') | (sample['NED_TYPE']=='QSO')

            e_sources = sample[(sample['T']<=-3) & ~object_condition]
            l_sources = sample[(sample['T']>-3) & (sample['T']<=0) & ~object_condition]         #Divide samples in Elliptical, Lenticular and Spiral by T value
            s_sources = sample[(sample['T']>0) & (sample['T']!=no_data) & ~object_condition]

            no_type_sources = sample[(sample['T']==no_data) & ~object_condition]                #Sample without T values
            w1w2_no_type = no_type_sources['W1mag'] - no_type_sources['W2mag']                  #Mean value of colors for previous sample
            w2w3_no_type = no_type_sources['W2mag'] - no_type_sources['W3mag']
            z_no_type = no_type_sources['Z']                                                    

            e_w1_kcorrected = wm.w1_k_corrected(lookup_table=e_table,w1=w1[e_sources.index],z=z[e_sources.index])
            l_w1_kcorrected = wm.w1_k_corrected(lookup_table=l_table,w1=w1[l_sources.index],z=z[l_sources.index])               #W1 k-corrected 
            s_w1_kcorrected = wm.w1_k_corrected(lookup_table=s_table,w1=w1[s_sources.index],z=z[s_sources.index])

            e_w2w3_kcor = wm.get_correction_factor(lookup_table=e_table, redshift=e_sources['Z'], correction_factor='W2-W3')
            l_w2w3_kcor = wm.get_correction_factor(lookup_table=l_table, redshift=l_sources['Z'], correction_factor='W2-W3')     #W2-W3 k-correction factor
            s_w2w3_kcor = wm.get_correction_factor(lookup_table=s_table, redshift=s_sources['Z'], correction_factor='W2-W3')

            e_w1w2_kcor = wm.get_correction_factor(lookup_table=e_table, redshift=e_sources['Z'], correction_factor='W1-W2')
            l_w1w2_kcor = wm.get_correction_factor(lookup_table=l_table, redshift=l_sources['Z'], correction_factor='W1-W2')     #W1-W2 k-correction factor
            s_w1w2_kcor = wm.get_correction_factor(lookup_table=s_table, redshift=s_sources['Z'], correction_factor='W1-W2')

            #Both k-correction factors for no type sample, using division by observed W2-W3 color
            factor_w2 = 171.787
            factor_w3 = 31.674

            w2w3_limit = -2.5*(np.log10(factor_w3/factor_w2) + 0.1)                             #Last value is the flux limit from Mateos et al. (2012)

            no_type_w1w2_kcor = [
                f.get_correction_factor(lookup_table=e_table, redshift=z, correction_factor='W1-W2') if (color_x<=w2w3_limit)
                else f.get_correction_factor(lookup_table=s_table, redshift=z, correction_factor='W1-W2') if (color_x>w2w3_limit) else 0
                for color_x,color_y,z in zip(w2w3_no_type,w1w2_no_type, z_no_type)
                ]
            no_type_w2w3_kcor = [
                f.get_correction_factor(lookup_table=e_table, redshift=z, correction_factor='W2-W3') if (color_x<=w2w3_limit)
                else f.get_correction_factor(lookup_table=s_table, redshift=z, correction_factor='W2-W3') if (color_x>w2w3_limit)  else 0
                for color_x,color_y,z in zip(w2w3_no_type,w1w2_no_type, z_no_type)
                ]

            no_type_w1_kcorrected = [
                f.w1_k_corrected(lookup_table=e_table,w1=w1[no_type_index],z=np.array([z]))[0] if (color_x<=w2w3_limit)
                else f.w1_k_corrected(lookup_table=s_table,w1=w1[no_type_index],z=np.array([z]))[0] if (color_x>w2w3_limit) else w1[no_type_index]
                for color_x,color_y,z,no_type_index in zip(w2w3_no_type,w1w2_no_type, z_no_type, no_type_sources.index)
                ]   

            e_w1abs_kcorrected = e_w1_kcorrected - f.distance_modulus_dist(dist[e_sources.index])[:,None]                     #W1 k-correction for e_sources
            l_w1abs_kcorrected = l_w1_kcorrected - f.distance_modulus_dist(dist[l_sources.index])[:,None]                     #W1 k-correction for l_sources
            s_w1abs_kcorrected = s_w1_kcorrected - f.distance_modulus_dist(dist[s_sources.index])[:,None]                     #W1 k-correction for s_sources
            no_type_w1abs_kcorrected = no_type_w1_kcorrected - f.distance_modulus_dist(dist[no_type_sources.index])[:,None]   #W1 k-correction for no_type_sources

            if np.shape(no_type_w1abs_kcorrected)==(0,0):
                no_type_w1abs_kcorrected = no_type_w1abs_kcorrected.reshape(0,number)

            w1w2_kcors[e_sources.index] = e_w1w2_kcor
            w1w2_kcors[l_sources.index] = l_w1w2_kcor                   #W1-W2 k-correctrion factors filled
            w1w2_kcors[s_sources.index] = s_w1w2_kcor
            w1w2_kcors[no_type_sources.index] = no_type_w1w2_kcor

            w2w3_kcors[e_sources.index] = e_w2w3_kcor
            w2w3_kcors[l_sources.index] = l_w2w3_kcor                   #W2-W3 k-correctrion factors filled
            w2w3_kcors[s_sources.index] = s_w2w3_kcor
            w2w3_kcors[no_type_sources.index] = no_type_w2w3_kcor

            w1abs[e_sources.index] = e_w1abs_kcorrected
            w1abs[l_sources.index] = l_w1abs_kcorrected                 #W1abs k-corrected filled
            w1abs[s_sources.index] = s_w1abs_kcorrected
            w1abs[no_type_sources.index] = no_type_w1abs_kcorrected
            
        else:
            w1abs_kcorrected = w1[sample.index] - f.distance_modulus_dist(dist[sample.index])[:,None]
            w1abs[sample.index] = w1abs_kcorrected

            w1w2_kcors[sample.index] = 0
            w2w3_kcors[sample.index] = 0                                        #For sources with z>3, no k-correction is applied
    else:
        continue

AttributeError: 'Table' object has no attribute 'index'